In [3]:
pip install scapy

In [2]:
from scapy.all import * # Packet manipulation
import pandas as pd # Pandas - Create and Manipulate DataFrames
import numpy as np # Math Stuff (don't worry only used for one line :] )
import binascii # Binary to Ascii 
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

### Se leen los paquetes analizados por el catedratico

In [18]:
#num_of_packets_to_sniff = 100
#pcap = sniff(count=num_of_packets_to_sniff)
pcap = rdpcap('analisis_paquetes.pcap')

# rdpcap returns packet list
## packetlist object can be enumerated 
print(type(pcap))
print(len(pcap))
print(pcap)
pcap[0]

<class 'scapy.plist.PacketList'>
62
<analisis_paquetes.pcap: TCP:0 UDP:62 ICMP:0 Other:0>


<Ether  dst=80:37:73:96:9b:db src=88:e9:fe:6a:92:52 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=961 id=1 flags= frag=0 ttl=64 proto=17 chksum=0x52e6 src=10.1.10.53 dst=84.54.22.33 |<UDP  sport=53 dport=53 len=941 chksum=0xf60e |<DNS  id=12 qr=0 opcode=QUERY aa=0 tc=0 rd=1 ra=0 z=0 ad=0 cd=0 rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='google.com.' qtype=AAAA qclass=IN |> an=None ns=None ar=None |<Raw  load='�PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01b\x00\x00\x00�\x08\x06\x00\x00\x00(�TR\x00\x00:�IDATx��\t|T�����;K\x12\x08;\x08�\nE��$\x19��Ziߪ-�;N2��V�V�a�Z\x11!��\x01ܻik[�.ok�e�\x01\\��]�-���d�\x08�\x08��g���<��L\x12�$3wf��\x02<_\r�Y�r��y���\x1c\r\x18�8c�ew\r\x1e����\x13�iaK����u}Ųw�d��\x15�"`\x06:\x13�\x0b�\x0c\x16�\x12�&�9Ɛ�\x00���\x08\x111ҌQHP*�@\x04O4���7<y�\x0e.9���a2@a��&"�\x13\x11�c�kO�s@�9�i�4X�x�K�a!f��0w��3dx\x1d"���C(�}�0|q�*��\x0b�a!f�4��.�sKg\x10\x11]=9NԂ\x06��u���K_�e\x06*���\x19Px��%��\x03�3��\x1e"E?\x0f���+H�]Q�k����8�"�\x1d\x114v�2\x10���咯̸|�x.\\

### Exploración de paquetes capturados

In [22]:
#Recordemos que en los modelos de red, las capas inferiores encapsulan la información de las capas superiores
#Frames en paquetes, paquetes en segmentos (Ethernet, IP), modelo TCP/IP

ethernet_frame = pcap[0]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload #recupera la informacion de la capa 4

print("Informacion del frame:")
print(ethernet_frame.summary())

print("Informacion del paquete:")
print(ip_packet.summary())

print("Informacion del segmento:")
print(segment.summary())

print("Data:")
print(data.summary())

ethernet_frame.show()
data.show()

Informacion del frame:
Ether / IP / UDP / DNS Qry "b'google.com.'"  / Raw
Informacion del paquete:
IP / UDP / DNS Qry "b'google.com.'"  / Raw
Informacion del segmento:
UDP / DNS Qry "b'google.com.'"  / Raw
Data:
DNS Qry "b'google.com.'"  / Raw
###[ Ethernet ]### 
  dst       = 80:37:73:96:9b:db
  src       = 88:e9:fe:6a:92:52
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 961
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = 17
     chksum    = 0x52e6
     src       = 10.1.10.53
     dst       = 84.54.22.33
     \options   \
###[ UDP ]### 
        sport     = 53
        dport     = 53
        len       = 941
        chksum    = 0xf60e
###[ DNS ]### 
           id        = 12
           qr        = 0
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 1
           ra        = 0
           z         = 0
           ad        = 0
       

### Tipos de datos registrados con Scapy

In [23]:
print(type(ethernet_frame))
print(type(ip_packet))
print(type(segment))

# Packets can be filtered on layers ie) ethernet_frame[scapy.layers.l2.Ether]
ethernet_type = type(ethernet_frame)
ip_type = type(ip_packet)
tcp_type = type(segment)
print("Ethernet",pcap[ethernet_type])
print("IP", pcap[ip_type])
print("TCP", pcap[tcp_type])

# Scapy provides this via import statements
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.layers.inet import TCP, UDP

print("UDP", pcap[UDP])

<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.inet.IP'>
<class 'scapy.layers.inet.UDP'>
Ethernet <Ether from analisis_paquetes.pcap: TCP:0 UDP:62 ICMP:0 Other:0>
IP <IP from analisis_paquetes.pcap: TCP:0 UDP:62 ICMP:0 Other:0>
TCP <UDP from analisis_paquetes.pcap: TCP:0 UDP:62 ICMP:0 Other:0>
UDP <UDP from analisis_paquetes.pcap: TCP:0 UDP:62 ICMP:0 Other:0>


### Se guarda la info en un Data Set

In [25]:
# Obtenemos los campos de IP/TCP/UDP que seran las columnas de nuestro DF
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

#Concatenamos todos los campos
dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Creamos un DF en blanco con los campos como columnas
df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    # Obtenemos los valores de los campos de cada fila
    field_values = []
    # A;adimos todos los valores de los campos de IP al df
    for field in ip_fields:
        if field == 'options':
            # Obtenemos el numero de opciones del IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    
    field_values.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    # Agregamos el payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Agregamos la fila al df
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

print(df.head(5))

  version ihl tos   len     id flags frag  ttl proto chksum  ... dataofs  \
0       4   5   0   961      1          0   64    17  21222  ...    None   
1       4   5  32    84  58919          0  122    17  13836  ...    None   
2       4   5   0   975      1          0   64    17  21208  ...    None   
3       4   5  32    84  59493          0  122    17  13262  ...    None   
4       4   5   0  1012      1          0   64    17  21171  ...    None   

  reserved flags window chksum urgptr options payload  \
0     None  None   None  62990   None    None     933   
1     None  None   None  65061   None    None      56   
2     None  None   None  36378   None    None     947   
3     None  None   None  65063   None    None      56   
4     None  None   None  63846   None    None     984   

                                         payload_raw  \
0  b'\x00\x0c\x01\x00\x00\x01\x00\x00\x00\x00\x00...   
1  b'\x00\x0c\x81\x80\x00\x01\x00\x01\x00\x00\x00...   
2  b'\x00\x0c\x01\x00\x00\x01\x0

In [26]:
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,10.1.10.53,84.54.22.33,53,53
1,84.54.22.33,10.1.10.53,53,53
2,10.1.10.53,84.54.22.33,53,53
3,84.54.22.33,10.1.10.53,53,53
4,10.1.10.53,84.54.22.33,53,53
...,...,...,...,...
57,84.54.22.33,10.1.10.53,53,53
58,10.1.10.53,84.54.22.33,53,53
59,84.54.22.33,10.1.10.53,53,53
60,10.1.10.53,84.54.22.33,53,53


In [27]:
#Calcular estadisticas
print("# Top Source Address")
print(df['src'].describe(),'\n\n')

# Top Source Address
count             62
unique             3
top       10.1.10.53
freq              31
Name: src, dtype: object 




In [28]:
#¿Quien es la IP de origen mas frecuente?
frequent_address = df['src'].describe()['top']


# A que IPs les habla la IP mas frecuente
print(df[df['src'] == frequent_address]['dst'].unique(),"\n\n")

['84.54.22.33' '75.75.75.75'] 




In [29]:
print(df)

   version ihl tos   len     id flags frag  ttl proto chksum  ... dataofs  \
0        4   5   0   961      1          0   64    17  21222  ...    None   
1        4   5  32    84  58919          0  122    17  13836  ...    None   
2        4   5   0   975      1          0   64    17  21208  ...    None   
3        4   5  32    84  59493          0  122    17  13262  ...    None   
4        4   5   0  1012      1          0   64    17  21171  ...    None   
..     ...  ..  ..   ...    ...   ...  ...  ...   ...    ...  ...     ...   
57       4   5  32    84   8744          0  122    17  64011  ...    None   
58       4   5   0   969      1          0   64    17  21214  ...    None   
59       4   5  32    84   9344          0  122    17  63411  ...    None   
60       4   5   0   588      1          0   64    17  21595  ...    None   
61       4   5  32    84  10094          0  122    17  62661  ...    None   

   reserved flags window chksum urgptr options payload  \
0      None  None

In [30]:
from google.colab import files

df.to_csv('packets.csv', encoding = 'utf-8-sig') 
files.download('packets.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>